# Trabajo Práctico 1 - Matrices de Insumo Producto

## Enunciado

### Consigna 1 - 

Vamos a interpretar las posibles soluciones que podemos obtener del sistema $$ p = (I - A)^{-1} d = Ld $$ Para ello tendremos en cuenta que significa que una matriz sea inversible o no y como impacta esto en las soluciones de mi sistema de ecuaciones lineales.
<p style="text-align: left;">
a) \( (I - A) \ \text{es inversible} \)
</p>
Si la matriz (I - A) tiene inversa esto significa que existe la matriz de Leontief, luego existe solucion unica para este sistema.
<p></p>
<p style="text-align: left;">
b) \( (I - A) \ \text{es inversible y d = 0} \)
</p>
En este caso existe matriz de Leontief pero mi sistema resulta en $p = (I - A)^{-1}\cdot0 = L\cdot0 = 0$  Podemos decir que no existe demanda externa en este caso y que la produccion total es destinada a satisfacer la demanda interna.
<p></p>
<p style="text-align: left;">
c) \( (I - A) \ \text{no es inversible} \)
</p>
Al no existir inversa esto significa que no se puede calcular la matriz de Leontief. Esto tiene dos posibles conclusiones:  <p></p> 
  . El sistema no tiene solucion. <p></p> 
  . Existen infinitas soluciones debido que hay dependencia lineal entre las ecuaciones de mi sistema. Esto puede ocurrir debido a una relacion entre sectores que determinan que no hay unica solucion.
<p style="text-align: left;">
d) \( (I - A) \ \text{no es inversible y d = 0} \)
</p>    
Debido a que no existe matriz de Leontief y que d = 0, para este caso el sistema admite infinitas soluciones para p.



### Consigna 2 -  

In [1]:
import numpy as np
from scipy.linalg import solve_triangular

def calcularLU(A):
    m = A.shape[0]
    n = A.shape[1]
    Ac = A.copy()
    P = np.eye(n)
    
    if m != n:
        print('Matriz no cuadrada')
        return

    L = np.eye(n) 
        
    for i in range(0, n):   
        max_row = np.argmax(np.abs(Ac[i:n, i])) + i
        if i != max_row:
            # Intercambia las filas en Ac
            Ac[[i, max_row], :] = Ac[[max_row, i], :]
            
            # Intercambia las filas en P
            P[[i, max_row], :] = P[[max_row, i], :]
            
            # Intercambia las filas en L hasta la columna i (excluyendo la diagonal)
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]
            
        for j in range(i+1, n):
            piv = Ac[j][i] / Ac[i][i]
            Ac[j] = Ac[j] - piv * Ac[i]
            L[j][i] = piv

    U = Ac
    return L, U, P

---

In [2]:

def inversaLU(L, U, P):
    n = L.shape[0]
    M_identidad = []
    Inv = np.zeros((n, 0)) 

    if P is None:
        P = np.eye(n)  # Si no se proporciona P, se usa la identidad

    # Crear la matriz identidad
    for i in range(n):
        canonico = np.zeros(n)
        canonico[i] = 1
        M_identidad.append(canonico)

    # Resolver el sistema para cada columna de la identidad permutada
    for i in range(n):
        b = P[:, i]  # Columna i de la matriz de permutación
        x = solve_triangular(L, b, lower=True)
        y = solve_triangular(U, x, lower=False)
        Inv = np.column_stack((Inv, y))  # Agregar la columna de la inversa

    return Inv


### Consigna 3 -  

$$
A = \begin{pmatrix}
0.3 & 0.0 & 0.1 \\
0.05 & 1.0 & 0.2 \\
0.1 & 0.15 & 0.1
\end{pmatrix}
, \quad
d = \begin{pmatrix}
100 \\
100 \\
300
\end{pmatrix}
$$


In [6]:
A = np.array([[0.3,0.0,0.1],[0.05,1.0,0.2],[0.1,0.15,0.1]])
d = np.array([100,100,300])

L,U,P = calcularLU(np.eye(3) - A)

print("P = " + str(inversaLU(L, U, P)@d))


P = [   68.96551724 -5149.42528736  -517.24137931]


a)
En el contexto de un modelo de insumo-producto como el de Leontief, el vector de producción 𝑃
indica la cantidad de bienes o servicios que deben producirse en cada sector para satisfacer la demanda interna y externa.
En nuestro caso podemos explicar los requerimientos de cada sector de P de la siguiente forma :

Primer sector (68.97): El primer valor del vector indica que el primer sector debe producir 68.97 unidades. Esta cantidad satisface tanto la demanda externa como los insumos que requieren los otros sectores. Por lo tanto este sector se encuentra en superávit.

Segundo sector (-5149.43): Un valor negativo sugiere que la producción en el segundo sector no solo es insuficiente para cubrir la demanda externa, sino que probablemente es un insumo significativo para otros sectores. Este déficit indica que se requiere una gran cantidad de productos del segundo sector como insumos para producir en otros sectores, lo que crea una dependencia significativa. Por lo tanto este sector se encuentra en déficit

Tercer sector (-517.24): Similar al segundo sector, este valor negativo implica un consumo neto de insumos del tercer sector. El sistema consume 517.24 unidades del tercer sector para satisfacer las demandas de otros sectores o la demanda final. Por lo tanto este sector se encuentra en déficit.

b)
La caracteristica de la matriz de Leontief L = (I-A) que lleva a esta solucion unica, es que es inversible. Es decir, su determinante es distinto de cero, por lo que la economía descrita por la matriz A es autosuficiente y sostenible en su estructura. Esto implica que cada sector, aunque dependa de otros, es capaz de satisfacer la demanda externa y su demanda interna de insumos. Ademas, la estructura de las dependencias entre los sectores es tal que el sistema económico no cae en un ciclo de dependencia infinita entre sectores. Si el determinante fuera cero, indicaría que hay una dependencia excesiva o una redundancia en el sistema. Por ejemplo, un sector podría necesitar infinitas unidades de otro sector para producir, lo cual es inviable.

### Consigna 4 -  